<a href="https://colab.research.google.com/github/carlosinator/cil-sentiment/blob/main/gero_read_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%%bash
pip3 install transformers emoji==0.6.0 keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.5/576.5 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 128.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
!git clone https://github.com/carlosinator/cil-sentiment.git

Cloning into 'cil-sentiment'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 175 (delta 33), reused 28 (delta 25), pack-reused 128
Receiving objects: 100% (175/175), 445.90 KiB | 21.23 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras_nlp
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
import transformers
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, TFAutoModelForSequenceClassification
import matplotlib.pyplot as plt
import pickle
import re
import subprocess as sp
import os
from threading import Thread , Timer
import sched, time

import sys
sys.path.append("./cil-sentiment/models")
sys.path.append("./cil-sentiment/")
from gru_models import GRUModel, VGRUModel
import utils

# reproducibility
transformers.set_seed(0) # sets the seed in random, numpy, and tf

Using TensorFlow backend


In [5]:
!gsutil cp "gs://cil_2023/train_pos_full_preprocessed_without_duplicates.txt" .
!gsutil cp "gs://cil_2023/train_neg_full_preprocessed_without_duplicates.txt" .

model_name = "vinai/bertweet-base"
filename_train_pos = "train_pos_full_preprocessed_without_duplicates.txt"
filename_train_neg = "train_neg_full_preprocessed_without_duplicates.txt"

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

Copying gs://cil_2023/train_pos_full_preprocessed_without_duplicates.txt...
- [1 files][ 74.6 MiB/ 74.6 MiB]   10.9 MiB/s                                   
Operation completed over 1 objects/74.6 MiB.                                     
Copying gs://cil_2023/train_neg_full_preprocessed_without_duplicates.txt...
| [1 files][ 91.6 MiB/ 91.6 MiB]   11.1 MiB/s                                   
Operation completed over 1 objects/91.6 MiB.                                     


In [6]:
# read dataset
dataset_pos_pd = pd.read_table(filename_train_pos, sep='\r\n', header=None, names=['text'])
dataset_neg_pd = pd.read_table(filename_train_neg, sep='\r\n', header=None, names=['text'])
dataset_pos_pd['label'] = 0
dataset_neg_pd['label'] = 1
dataset_pd = pd.concat([dataset_pos_pd, dataset_neg_pd])

# shuffle
dataset_pd = dataset_pd.sample(frac=1, random_state=0).reset_index(drop=True)

# tokenize data set
tokenizer = AutoTokenizer.from_pretrained(model_name)
texts = tokenizer.batch_encode_plus(dataset_pd['text'].tolist(),
                                    padding=True, truncation=True,
                                    return_tensors='tf')

dataset = tf.data.Dataset.from_tensor_slices((dict(texts), dataset_pd['label']))


<ipython-input-6-6f8e9b745d48>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset_pos_pd = pd.read_table(filename_train_pos, sep='\r\n', header=None, names=['text'])
<ipython-input-6-6f8e9b745d48>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset_neg_pd = pd.read_table(filename_train_neg, sep='\r\n', header=None, names=['text'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [76]:
# split training / validation
batch_size = 256 # * tpu_strategy.num_replicas_in_sync
AUTOTUNE = tf.data.experimental.AUTOTUNE

val_data_size = int(0.1 * len(dataset_pd.index))
test_data_size = int(0.1 * len(dataset_pd.index))
train_data_size = len(dataset_pd.index) - val_data_size - test_data_size
val_ds = dataset.take(val_data_size).batch(batch_size, drop_remainder=True)
test_ds = dataset.take(test_data_size).batch(batch_size, drop_remainder=True)
train_ds = dataset.skip(val_data_size).batch(batch_size, drop_remainder=True)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

# Train model for inference

In [7]:
import subprocess as sp
import os
from threading import Thread , Timer
import sched, time

gpu_hist = {
    "mib" : [],
    "percent" : [],
    "counter" : 0,
}

def get_gpu_memory():

    out = !nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv

    float_pattern = r'\d+\.\d+|\d+'
    numbers = re.findall(float_pattern, out[1])

    # Extract the MiB value and the percentage
    mib_val = float(numbers[0])
    perc = float(numbers[1])

    gpu_hist["counter"] += 1

    # print(mib_val, perc)

    gpu_hist["mib"].append(mib_val)
    gpu_hist["percent"].append(perc)

    return


def track_gpu_mem(interval=5.0, dictname="gpu_hist.pkl"):
    """
        This function calls itself every 5 secs and print the gpu_memory.
    """
    thd = Timer(interval, track_gpu_mem, [interval, dictname])
    thd.start()
    get_gpu_memory()

    gpu_hist["interval"] = interval

    with open(dictname, 'wb') as f:
      pickle.dump(gpu_hist, f)

    return thd

In [72]:
# custom model
class CustomModel(tf.keras.Model):
    def __init__(self, backbone_model_name, num_rnn_units):
        super(CustomModel, self).__init__()

        # Load transformer model config
        config = AutoConfig.from_pretrained(backbone_model_name, output_hidden_states=True)

        # Load the backbone transformer model
        self.backbone_model = TFAutoModel.from_pretrained(backbone_model_name, config=config)

        # Freeze the backbone model
        self.backbone_model.trainable = False

        #NOTE: parameters are fused across all layers!!! (so only a single RNN, no joiner, etc.)

        # Define the RNN layer (i.e. a layer that has automatically as much cells as needed for the input sequence)
        self.rnn = tf.keras.layers.GRU(num_rnn_units, return_sequences=True)

        # Add a correction layer that will learn the correction from the RNN output
        #  (i.e. the one that transforms the output of the RNN to the hidden_size of each layer -> this is like giving the RNN a head which actually outputs corrections)
        self.correction_layer = tf.keras.layers.Dense(self.backbone_model.config.hidden_size, use_bias=False)

        # Using GlobalAveragePooling1D to convert the sequence of corrected hidden states into a single vector
        self.pooling = tf.keras.layers.GlobalAveragePooling1D()

        # Define the final Dense layer for binary classification
        self.classifier = tf.keras.layers.Dense(2, activation='softmax') # either positive or negative

    def call(self, inputs, training=False):
        # Pass the inputs through the backbone model
        outputs = self.backbone_model(inputs, training)

        # Get the hidden states from the outputs
        hidden_states = outputs.hidden_states
        # in particular, if we take the last_hidden_state, we can use it to gather some information about the processed sequence (like the input sequence_length)
        last_hidden_state = outputs.last_hidden_state

        #batch_size, sequence_length, hidden_size = tf.shape(last_hidden_state) # see https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_outputs.BaseModelOutput

        # About the corrections:
        # - the corrections for a single layer are a vector of size sequence_length

        # Initialize corrections h with zeros
        # NOTE: h is always the correction for a single layer of the backbone model! (we start with the first one, initialized to zeros)
        # for every timestep in a sequence of length sequence_length there are hidden_size many hidden states -> every hidden state needs a correction
        #h = tf.zeros_like(last_hidden_state[0]) # has shape (sequence_length, hidden_size), the first batch is like an example for the shape of all batches
        h = tf.zeros_like(last_hidden_state) # has shape (batch_size, sequence_length, hidden_size)

        # Iterate over the hidden states
        for i, hidden_state in enumerate(hidden_states): #iterate over all layers of the backbone
            # Compute RNN output using the hidden state and the previous corrections
            #rnn_output = self.rnn(tf.concat([hidden_state, h], axis=-1))
            rnn_output = self.rnn(tf.concat([hidden_state, h], axis=-1))

            # apply a transformation to get actual correction values
            h = self.correction_layer(rnn_output)

        # Get the corrected last hidden state
        corrected_last_hidden_state = last_hidden_state + h

        # Using pooling before passing the sequence through the classifier
        pooled_output = self.pooling(corrected_last_hidden_state)

        # Classify the corrected last hidden state of the backbone
        return self.classifier(pooled_output)

In [77]:
USE_MODEL = "read_gero"
READ_GRU_UNITS = 16
LEARNING_RATE = 1e-3
EPOCHS = 5
model = CustomModel(backbone_model_name=model_name, num_rnn_units=READ_GRU_UNITS)
model.compile(
          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
          optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, amsgrad=False, clipnorm=None),
          metrics=[tf.metrics.SparseCategoricalAccuracy()]
      )

history = {}

run_name = "inference_" + USE_MODEL + "_fullmodel_" + f"lr{LEARNING_RATE}_ep{EPOCHS}_batchs={batch_size}_gru_units={READ_GRU_UNITS}"
hd_name = run_name + "_dict.pkl"
gpu_hist_name = "gpu_hist_" + run_name + ".pkl"

memory.used [MiB], utilization.gpu [%]
9479 MiB, 0 %


Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [78]:
tf.keras.backend.clear_session()

memory.used [MiB], utilization.gpu [%]
9479 MiB, 0 %
memory.used [MiB], utilization.gpu [%]
9479 MiB, 0 %


In [79]:
callback = tf.keras.callbacks.EarlyStopping(patience=3) # early stopping def
gpu_mem_proc = track_gpu_mem(10.0, dictname=gpu_hist_name) # initialize gpu tracking

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1,
    #callbacks=[callback]
    )

gpu_mem_proc.join()

7981/7981 [==============================] - 1455s 180ms/step - loss: 0.2714 - sparse_categorical_accuracy: 0.8830 - val_loss: 0.2496 - val_sparse_categorical_accuracy: 0.8940
Epoch 2/10
7981/7981 [==============================] - 1430s 179ms/step - loss: 0.2574 - sparse_categorical_accuracy: 0.8899 - val_loss: 0.2491 - val_sparse_categorical_accuracy: 0.8946
Epoch 3/10
7981/7981 [==============================] - 1429s 179ms/step - loss: 0.2537 - sparse_categorical_accuracy: 0.8916 - val_loss: 0.2450 - val_sparse_categorical_accuracy: 0.8960
Epoch 4/10
7981/7981 [==============================] - 1429s 179ms/step - loss: 0.2515 - sparse_categorical_accuracy: 0.8928 - val_loss: 0.2439 - val_sparse_categorical_accuracy: 0.8965
Epoch 5/10
7981/7981 [==============================] - 1430s 179ms/step - loss: 0.2497 - sparse_categorical_accuracy: 0.8935 - val_loss: 0.2427 - val_sparse_categorical_accuracy: 0.8971
Epoch 6/10
7981/7981 [==============================] - 1427s 179ms/step - l

KeyboardInterrupt: ignored

In [80]:
# save model
model.save(run_name)
# save training history
with open(hd_name, 'wb') as f:
  pickle.dump(history, f)

#Note: the gpu history should already been stored at gpu_hist_name

!gsutil cp -r {run_name + "/"} "gs://cil_2023/models/"
!gsutil cp {hd_name} "gs://cil_2023/models/"
!gsutil cp {gpu_hist_name} "gs://cil_2023/models/"

Copying file://inference_read_gero_fullmodel_lr0.001_ep10_batchs=256_gru_units=16/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://inference_read_gero_fullmodel_lr0.001_ep10_batchs=256_gru_units=16/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://inference_read_gero_fullmodel_lr0.001_ep10_batchs=256_gru_units=16/variables/variables.index [Content-Type=application/octet-stream]...
|
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://inference_read_gero_fullmodel_lr0.001_ep10_batchs=256_gru_units=16/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature 

# Retrieve model for inference

In [ ]:
# uncomment if trained_model is not on local machine
# !gsutil cp -r {"gs://cil_2023/models/" + run_name} .
trained_model = tf.keras.models.load_model(run_name)
trained_model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  134309376 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592130    
 ificationHead)                                                  
                                                                 
Total params: 134901506 (514.61 MB)
Trainable params: 134901506 (514.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Test accuracy

In [ ]:
preds = trained_model.predict(test_ds)

if USE_MODEL == "basemodel":
  preds = tf.keras.layers.Softmax()(preds)